In [1]:
from multiprocessing import Pool
import pickle
import gzip
import numpy as np
import os, os.path
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from PIL import Image, ImageOps
from itertools import repeat
from sklearn.model_selection import train_test_split
import re
import keras
from keras import backend as K
from data_manager import DataManager, load_img, list_images
from data_generator import CustomDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from datetime import datetime
import tensorflow
from model import UNet
import tensorflow.keras.activations as activations
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_addons as tfa
import skimage
import scipy.spatial.distance as spdist
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim

/home/freshpate/miniconda3/envs/cern/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:37: UserWarning: You are currently using a nightly version of TensorFlow (2.9.0-dev20211227). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  warnings.warn(


In [2]:
gpus = tensorflow.config.experimental.list_physical_devices('GPU')

print("Available GPUs: ", [gpu.name for gpu in gpus])
gpu_index = 0
print(f"Running on GPU {gpu_index}")
tf.config.experimental.set_visible_devices(devices=gpus[gpu_index], device_type='GPU')
tf.config.experimental.set_memory_growth(device=gpus[gpu_index], enable=True)

Available GPUs:  ['/physical_device:GPU:0']
Running on GPU 0


In [3]:
BASE = os.getcwd()
manager = DataManager()
BATCH_SIZE = 6
EPOCHS = 200

In [4]:
def mask_not_blank(mask):
    return sum(mask.flatten()) > 0

def plot_image(img, title=None):
    plt.figure(figsize=(7,7))
    plt.title(title)
    plt.imshow(img)
#     plt.show()

def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

def resize2SquareKeepingAspectRation(img, size, interpolation):
    h, w = img.shape[:2]
    c = None if len(img.shape) < 3 else img.shape[2]
    if h == w: 
        return cv2.resize(img, (size, size), interpolation)
    if h > w: 
        dif = h
    else:     
        dif = w
    x_pos = int((dif - w)/2.)
    y_pos = int((dif - h)/2.)
    if c is None:
        mask = np.zeros((dif, dif), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w] = img[:h, :w]
    else:
        mask = np.zeros((dif, dif, c), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w, :] = img[:h, :w, :]
    return cv2.resize(mask, (size, size), interpolation)

def grays_to_RGB(img):
    # turn image into grayscale RGB
    return np.array(Image.fromarray(img).convert("RGB"))

def save_img(img, img_idx, path, pid, is_mask=False):
    filename = path + '/' + str(pid) + '_' + str(img_idx) 
    if is_mask: 
        filename += '_mask.png' 
        img = np.asarray(img, dtype="uint8") # convert bool mask into uint8 so cv2 doesn't scream
    else:
        filename += '.png'
        img = grays_to_RGB(img)
    
    cv2.imwrite(filename, img)

def make_dir(path):
    try:
        os.mkdir(path)
    except OSError:
        print (f"Creation of the directory {path} failed", end='\r')

def gen_dataset(imgs, dataset, pid, labels=None, typeof_dataset=None):
    output_dir = BASE + '/data/'+dataset+'/'
    if os.path.isdir(output_dir) is False:
        make_dir(output_dir)
    if typeof_dataset is not None: # this is only for train
        output_dir+=typeof_dataset #+ '/'
        if os.path.isdir(output_dir) is False:
            make_dir(output_dir)
    
    for i, img in enumerate(imgs):
        save_img(img, i, output_dir, pid)
        if labels is not None: # this is only for train
            save_img(labels[i], i, output_dir, pid, is_mask=True)
    
def list_images(directory, ext='jpg|jpeg|bmp|png|tif'):
    return [os.path.join(directory, f) for f in os.listdir(directory)
            if os.path.isfile(os.path.join(directory, f)) and re.match('([\w]+\.(?:' + ext + '))', f)]

def preprocess(img, denoise=False):
    """
    Preprocess step after image augmentation, and before feeding into conv net.
    """
    if denoise:
        img = cv2.fastNlMeansDenoising(img, h=7)
    
    img = resize2SquareKeepingAspectRation(img, DataManager.EX_IMG_TARGET_COLS, cv2.INTER_AREA)
    img = np.expand_dims(img, axis=-1)
    return img


def transform(img, mask, augment=True):
    """
    Transforms an (img, mask) pair with same augmentation params
    """
    if augment:
        pass
        #img, mask = augmenter.augment_batch(np.array([img, mask]), same_transform=True)
    img = preprocess(img)
    mask = preprocess(mask).astype('float32')
    return np.array([img]), np.array([mask])

In [5]:
def prind_de_pula(images, labels):
    sim=dict()
    for i in tqdm(range(len(images))):
        for j in range(len(images)):
            if i!=j:
                if tuple(sorted((i,j))) not in sim.keys():
                    simil=ssim(images[i], images[j])
                    if abs(simil)>0.90:
                        sim[tuple(sorted((i,j)))]=simil
    print(sim)
    return 0
    conf=[]
    for i in tqdm(sim.keys()):
        if sim[i]>0.99:
            msk1=cv2.imread(img.mask_path.iloc[i[0]],cv2.IMREAD_GRAYSCALE)
            msk2=cv2.imread(img.mask_path.iloc[i[1]],cv2.IMREAD_GRAYSCALE)
            if msk1.any()!=msk2.any():
                conf.append(i)

In [6]:
# train_data = load_zipped_pickle("data/train.pkl")
# test_data = load_zipped_pickle("data/test.pkl")
# for data in tqdm(train_data, total=len(train_data)):
#     imgs = data['video'].T
#     typeof_ds = data['dataset']
#     labels = data['label'].T
#     pacient = data['name']
#     prind_de_pula(imgs, labels)
#     print(data['frames'])
#     break
#     gen_dataset(imgs, "train", pacient, labels, typeof_ds)

# for data in tqdm(test_data, total=len(test_data)):
#     imgs = data['video'].T
#     pacient = data['name']
#     gen_dataset(imgs, "test", pacient)
# min_w = 1000
# max_w = 0
# min_h = 1000
# max_h = 0
# for data in tqdm(test_data, total=len(test_data)):
#     imgs = data['video'].T
#     pacient = data['name']
#     if min_w > imgs.shape[1]:
#         min_w = imgs.shape[1]
#     if max_w < imgs.shape[1]:
#         max_w = imgs.shape[1]
#     if min_h > imgs.shape[2]:
#         min_h = imgs.shape[2]
#     if max_h < imgs.shape[2]:
#         max_h = imgs.shape[2]

# print("Min width: {} Max width: {}".format(min_w, max_w))
# print("Min height: {} Max height: {}".format(min_h, max_h))

In [ ]:
manager.create_train_data()

In [ ]:
manager.create_test_data()

In [7]:
X_train, X_val, y_train, y_val = manager.load_train_val_data("expert")

In [ ]:
X = np.concatenate([X_train, X_val], axis=0)
y = np.concatenate([y_train, y_val], axis=0)

In [8]:
datagen = CustomDataGenerator(
    X_train,
    y_train,
    (224, 224),
    preprocess,
    batch_size=8,
    shuffle=True,
    seed=42,
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rotation_range=5, #degrees
    width_shift_range=10, #pixels, if <1 fraction
    height_shift_range=10,
    horizontal_flip=False,
    shear_range=5,
    rescale=1./255)
    #preprocessing_function=preprocess)
datagen.generator.fit(X_train)

In [ ]:
def plot_image_with_mask(img, mask, title=None):
    """
    this does the same thing as plot_separate but plots only the overlayed img
    """
    # returns a copy of the image with edges of the mask added in red
    img_mask = np.ma.masked_where(mask == False, mask)
    #img_box = np.ma.masked_where(box == False, box)
    plt.figure(figsize=(8,8))
    plt.imshow(img, cmap = 'gray', interpolation = 'none')
    plt.imshow(img_mask, cmap = 'jet', interpolation = 'none', alpha = 0.8)
    #plt.imshow(img_box, cmap = 'spring', interpolation = 'none', alpha = 0.5)
    plt.title(title)

In [ ]:
i = 0
for _ in range(len(X_train) // 2):
    batch_x, batch_y = datagen.next()
    yy = batch_y['output_2']
    if np.sum(yy) > 0:
        plot_image_with_mask(batch_x[0], batch_y['output_1'][0])
        #plot_image(batch_y['output_1'][0])
        #print(batch_y['output_1'][0].dtype)
        #print(batch_x[0].dtype)
        plt.show()
        i += 1
    if i == 10:
        break
    #break

In [9]:
datagen_val = CustomDataGenerator(
    X_val,
    y_val,
    (224, 224),
    preprocess,
    batch_size=8,
    shuffle=True,
    seed=42,
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rotation_range=5, #degrees
    width_shift_range=10, #pixels, if <1 fraction
    height_shift_range=10,
    horizontal_flip=False,
    shear_range=5,
    rescale=1./255)
    #preprocessing_function=preprocess)
datagen_val.generator.fit(X_val)

In [ ]:
x, y = datagen.next()

In [ ]:
print(y)

In [10]:
net = UNet(True)
net.build((None, 224, 224, 1))
net.summary()
run_id = str(datetime.now())
model_checkpoint = ModelCheckpoint('./results/net', monitor='val_loss', save_best_only=True, save_weights_only=True)
tb = TensorBoard(log_dir='./logs/{}'.format(run_id), histogram_freq=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=4, min_lr=1e-6)
print('Training on model')
net.compile(optimizer='adam',
                  loss={'output_1': 'binary_crossentropy', 'output_2': 'binary_crossentropy'},
                  metrics={'output_1': tf.keras.metrics.IoU(num_classes=2, target_class_ids=[0]), 'output_2': 'acc'},
                  loss_weights={'output_1': 1, 'output_2': 0.5})

2021-12-30 20:11:48.617933: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-30 20:11:49.389240: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5215 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "u_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 224, 224, 32)      9760      
                                                                 
 sequential_1 (Sequential)   (None, 112, 112, 64)      55808     
                                                                 
 sequential_2 (Sequential)   (None, 56, 56, 128)       222208    
                                                                 
 sequential_3 (Sequential)   (None, 28, 28, 256)       886784    
                                                                 
 sequential_4 (Sequential)   (None, 14, 14, 512)       3543040   
                                                                 
 conv2d_10 (Conv2D)          multiple                  100353    
                                                                 
 aux_output (Flatten)        multiple                  0     

In [11]:
net.fit(datagen, validation_data=datagen_val, epochs=2, steps_per_epoch=len(X_train)//8, validation_steps=X_val.shape[0]//8, callbacks=[model_checkpoint, reduce_lr, tb])

Epoch 1/2


2021-12-30 20:12:10.662768: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8300


 35/163 [=====>........................] - ETA: 3:49 - loss: 0.4708 - output_1_loss: 0.1857 - output_2_loss: 0.5703 - output_1_io_u: 0.9999 - output_2_acc: 0.9643

KeyboardInterrupt: 

In [ ]:
model.fit_generator(train_generator, validation_data=val_generator, validation_steps=X_val.shape[0],
                     steps_per_epoch=X_train.shape[0], epochs=EPOCHS, verbose=2,
                     callbacks=[model_checkpoint, reduce_lr, tb], max_queue_size=1000)